In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score,auc
from lightgbm import LGBMClassifier


In [ ]:
train_data = pd.read_csv('../input/janatahack-crosssell-prediction-dataset/train.csv',index_col = 'id')
test_data = pd.read_csv('../input/janatahack-crosssell-prediction-dataset/test.csv',index_col = 'id')
combined = [train_data,test_data]

In [ ]:
train_data.head()

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.info()

In [ ]:
train_data.describe()

In [ ]:
cat_features = train_data.select_dtypes(['object']).columns
cat_features

In [ ]:
for col in cat_features:
    print(train_data[col].nunique())

In [ ]:
le = LabelEncoder()
for col in cat_features:
    train_data[col] = le.fit_transform(train_data[col])
    test_data[col] = le.fit_transform(test_data[col])

In [ ]:
train_data.info()

In [ ]:
train_data.describe()

In [ ]:
plt.figure(figsize = (9,6))
sns.set_style('ticks')
sns.countplot('Gender', hue = 'Response',data = train_data)
plt.show()

In [ ]:
train_data['Age'].nunique()

In [ ]:
plt.figure(figsize = (9,6))
sns.set_style('ticks')
sns.violinplot('Response', 'Age',data = train_data)
plt.show()

In [ ]:
sc = StandardScaler()
col = 'Age'
train_data[col] = sc.fit_transform(train_data[col].values.reshape(-1,1))
test_data[col] = sc.transform(test_data[col].values.reshape(-1,1))

In [ ]:
train_data['Driving_License'].nunique()

In [ ]:
train_data[train_data['Driving_License'] == 1]['Response'].count()

In [ ]:
plt.figure(figsize = (9,6))
sns.set_style('ticks')
sns.countplot('Driving_License', hue = 'Response',data = train_data)
plt.show()

In [ ]:
train_data['Region_Code'].nunique()

In [ ]:
plt.figure(figsize = (9,6))
sns.set_style('ticks')
sns.violinplot('Response', 'Region_Code',data = train_data)
plt.show()

In [ ]:
train_data['Previously_Insured'].nunique()

In [ ]:
plt.figure(figsize = (9,6))
sns.set_style('ticks')
sns.countplot('Previously_Insured', hue = 'Response',data = train_data)
plt.show()

In [ ]:
plt.figure(figsize = (9,6))
sns.set_style('ticks')
sns.countplot('Vehicle_Age', hue = 'Response',data = train_data)
plt.show()

In [ ]:
train_data['Vehicle_Damage'].nunique()

In [ ]:
plt.figure(figsize = (9,6))
sns.set_style('ticks')
sns.countplot('Vehicle_Damage', hue = 'Response',data = train_data)
plt.show()

In [ ]:
train_data['Annual_Premium'].nunique()

In [ ]:
plt.figure(figsize = (9,6))
sns.set_style('ticks')
sns.violinplot('Response', 'Annual_Premium',data = train_data)
plt.show()

In [ ]:
# dataset = train_data
# index = dataset[dataset['Annual_Premium'] >= 115000]['Annual_Premium'].index.values
# dataset.drop(index,inplace = True)

In [ ]:
sc = StandardScaler()
col = 'Annual_Premium'
train_data[col] = sc.fit_transform(train_data[col].values.reshape(-1,1))
test_data[col] = sc.transform(test_data[col].values.reshape(-1,1))

In [ ]:
plt.figure(figsize = (9,6))
sns.set_style('ticks')
sns.violinplot('Response', 'Annual_Premium',data = train_data)
plt.show()

In [ ]:
train_data['Policy_Sales_Channel'].nunique()

In [ ]:
mms = MinMaxScaler()
col = 'Policy_Sales_Channel'
train_data[col] = mms.fit_transform(train_data[col].values.reshape(-1,1))
test_data[col] = mms.transform(test_data[col].values.reshape(-1,1))

In [ ]:
sc = StandardScaler()
col = 'Policy_Sales_Channel'
train_data[col] = sc.fit_transform(train_data[col].values.reshape(-1,1))
test_data[col] = sc.transform(test_data[col].values.reshape(-1,1))

In [ ]:
plt.figure(figsize = (9,6))
sns.set_style('ticks')
sns.violinplot('Response', 'Policy_Sales_Channel',data = train_data)
plt.show()

In [ ]:
train_data['Vintage'].nunique()

In [ ]:
plt.figure(figsize = (9,6))
sns.set_style('ticks')
sns.violinplot('Response', 'Vintage',data = train_data)
plt.show()

In [ ]:
train_data.head()

In [ ]:
plt.figure(figsize = (9,6))
sns.heatmap(train_data.corr(),annot=True,cbar = True)
plt.show()

In [ ]:
y = train_data.loc[:,'Response']
X = train_data.drop('Response',axis = 1)
X

In [ ]:

x_train,x_val,y_train,y_val = train_test_split(X,y,test_size = 0.2,random_state = 42)


In [ ]:
model = LGBMClassifier(boosting_type='gbdt',n_estimators=500,depth=10,learning_rate=0.04,objective='binary',metric='auc',is_unbalance=True,
                 colsample_bytree=0.5,reg_lambda=2,reg_alpha=2,random_state=294,n_jobs=-1)

clf = model.fit(x_train,y_train)

In [ ]:
roc_auc_score(y_train,clf.predict(x_train))

In [ ]:
roc_auc_score(y_val,clf.predict_proba(x_val)[:,1])

In [ ]:
y_pred = clf.predict_proba(test_data)[:,1]

In [ ]:
sub = pd.DataFrame(y_pred,columns = ['Response'],index = [i + 381110 for i in range(test_data.shape[0])])

sub.index.name = 'id'
sub.tail()

In [ ]:
sub.to_csv('sub.csv')